In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

import lightgbm

In [2]:
# set options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 데이터 확인 및 전처리

In [3]:
vote_df = pd.read_csv("C:\\Users\\eunseo\\Desktop\\Git\\AI-project\\data\\train.csv", encoding="utf-8")
print("dataset shape: ", vote_df.shape)
vote_df.sample(10)

dataset shape:  (36425, 78)


,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,age_group,education,engnat,familysize,gender,hand,married,race,religion,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
9321,45506,1.0,511,5.0,705,5.0,1141,1.0,906,3.0,2647,1.0,545,1.0,673,1.0,1108,3.0,784,4.0,1339,2.0,1415,5.0,772,5.0,1169,2.0,642,3.0,1548,2.0,1456,3.0,981,2.0,465,4.0,1023,5.0,1128,10s,2,1,4,Male,3,1,White,Agnostic,1,6,1,6,0,5,2,5,0,3,3,1,0,1,0,1,1,0,1,1,0,1,1,0,1,0,1,1
11152,15299,2.0,1257,1.0,1155,2.0,1366,2.0,1274,5.0,1693,2.0,974,4.0,752,1.0,113,4.0,5938,3.0,1573,5.0,1055,5.0,1311,3.0,973,4.0,10226,2.0,959,2.0,1416,3.0,729,4.0,2532,2.0,740,1.0,1681,10s,2,1,4,Male,1,1,White,Agnostic,5,5,4,2,1,0,0,4,1,3,2,0,0,1,0,1,1,0,1,1,0,1,1,0,1,1,1,1
26977,17414,1.0,745,1.0,962,4.0,762,2.0,876,2.0,2114,1.0,489,3.0,1098,1.0,2200,2.0,2057,3.0,2581,2.0,3053,5.0,1206,5.0,899,1.0,498,2.0,3908,1.0,759,2.0,1052,1.0,1540,4.0,1099,4.0,1927,20s,3,1,3,Male,1,1,White,Atheist,2,1,4,6,2,1,4,1,0,4,2,1,0,0,0,1,1,0,1,1,0,1,1,0,1,0,1,1
33335,14483,2.0,736,2.0,823,5.0,2153,5.0,131,2.0,254,5.0,716,5.0,1747,4.0,2190,2.0,5509,5.0,9824,5.0,3244,5.0,3825,4.0,5764,3.0,823,4.0,5464,5.0,2214,3.0,1324,3.0,2663,4.0,3268,5.0,4330,10s,4,2,2,Male,1,1,Asian,Hindu,1,0,2,4,1,2,3,2,2,6,3,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0,1,1
7762,7033,4.0,564,2.0,1107,5.0,756,2.0,1083,2.0,1843,2.0,607,1.0,689,1.0,1736,3.0,1709,4.0,1885,3.0,2075,4.0,1389,2.0,1095,1.0,644,4.0,695,4.0,831,2.0,651,2.0,1093,3.0,754,4.0,1129,20s,4,1,2,Female,2,1,White,Atheist,5,2,2,0,2,2,4,3,5,5,1,1,0,0,0,1,1,0,0,1,1,1,1,0,1,0,1,1
13536,27045,3.0,603,1.0,1090,2.0,1288,1.0,1130,5.0,881,5.0,581,5.0,807,1.0,1045,5.0,1088,2.0,1825,5.0,1208,4.0,1247,1.0,728,5.0,650,1.0,1150,1.0,1249,1.0,541,5.0,1361,1.0,704,1.0,1302,60s,2,1,1,Female,1,2,White,Christian_Protestant,6,0,6,5,6,0,0,0,3,7,1,1,0,0,0,0,1,0,1,1,0,0,1,0,1,1,1,1
15493,33213,2.0,688,2.0,1988,5.0,506,2.0,1071,2.0,559,1.0,481,1.0,786,4.0,732,2.0,1507,4.0,956,4.0,951,5.0,797,4.0,226,1.0,605,4.0,1614,4.0,669,2.0,437,2.0,855,5.0,542,5.0,657,20s,3,1,1,Female,1,1,Other,Other,4,1,2,2,0,1,5,0,4,3,2,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1
22445,11479,1.0,515,2.0,1582,4.0,719,3.0,1390,3.0,1027,2.0,588,4.0,637,1.0,1940,4.0,1326,3.0,1706,3.0,857,4.0,2162,3.0,301,1.0,720,1.0,820,2.0,2643,2.0,454,1.0,712,2.0,613,5.0,1231,10s,2,1,3,Male,1,1,White,Agnostic,2,3,2,5,1,4,3,2,0,6,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1
12507,19187,2.0,451,3.0,1204,5.0,677,1.0,887,1.0,727,4.0,590,2.0,578,4.0,1094,2.0,1117,5.0,2278,5.0,1426,5.0,792,4.0,1176,4.0,528,4.0,2782,3.0,722,2.0,789,1.0,1821,3.0,729,4.0,850,30s,2,1,3,Male,1,3,White,Other,5,2,1,0,4,1,5,5,2,4,3,1,0,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1
16100,16814,2.0,506,2.0,1687,2.0,636,2.0,1231,3.0,1542,1.0,774,2.0,1759,1.0,1720,4.0,1389,2.0,1581,4.0,1736,2.0,1358,2.0,990,2.0,555,5.0,863,2.0,1026,4.0,1558,4.0,1137,2.0,1116,2.0,1013,10s,2,1,3,Female,1,1,Other,Christian_Catholic,3,2,5,1,0,1,2,2,4,2,3,0,0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,1


In [4]:
vote_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36425 entries, 0 to 36424
Data columns (total 78 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       36425 non-null  int64  
 1   QaA         36425 non-null  float64
 2   QaE         36425 non-null  int64  
 3   QbA         36425 non-null  float64
 4   QbE         36425 non-null  int64  
 5   QcA         36425 non-null  float64
 6   QcE         36425 non-null  int64  
 7   QdA         36425 non-null  float64
 8   QdE         36425 non-null  int64  
 9   QeA         36425 non-null  float64
 10  QeE         36425 non-null  int64  
 11  QfA         36425 non-null  float64
 12  QfE         36425 non-null  int64  
 13  QgA         36425 non-null  float64
 14  QgE         36425 non-null  int64  
 15  QhA         36425 non-null  float64
 16  QhE         36425 non-null  int64  
 17  QiA         36425 non-null  float64
 18  QiE         36425 non-null  int64  
 19  QjA         36425 non-nul

In [5]:
vote_df.describe()

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,QfA,QfE,QgA,QgE,QhA,QhE,QiA,QiE,QjA,QjE,QkA,QkE,QlA,QlE,QmA,QmE,QnA,QnE,QoA,QoE,QpA,QpE,QqA,QqE,QrA,QrE,QsA,QsE,QtA,QtE,education,engnat,familysize,hand,married,tp01,tp02,tp03,tp04,tp05,tp06,tp07,tp08,tp09,tp10,urban,voted,wf_01,wf_02,wf_03,wr_01,wr_02,wr_03,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
count,36425.000000,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,36425.000000,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,3.642500e+04,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.00000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000,36425.000000
mean,22740.586520,2.131942,9.392264e+02,2.897598,2.053925e+03,3.661414,1461.583775,1.750748,1.499025e+03,2.320494,1.897114e+03,2.167742,1.911749e+03,2.560220,1.287380e+03,2.316047,2.243674e+03,3.166699,3.508262e+03,3.574084,2.647131e+03,3.822704,1.910627e+03,4.330734,4.253209e+03,3.509979,1.665842e+03,2.635772,956.472258,3.213672,1410.536884,3.067756,1.632666e+03,2.842169,1.114106e+03,2.827948,2.632989e+03,3.431187,1.163574e+03,3.224104,2.059381e+03,2.548250,1.272752,2.621002,1.170487,1.387975,3.018174,2.61930,1.755717,3.025614,1.333342,2.462759,2.022704,3.509952,2.272340,4.317968,2.179574,0.453178,0.080494,0.163294,0.118600,0.709623,0.934468,0.208730,0.868717,0.895072,0.329087,0.938888,0.971613,0.202663,0.945532,0.393988,0.947536,0.926781
std,13118.110333,1.199763,1.372469e+04,1.568342,1.984665e+04,1.431303,8097.443081,1.046655,1.109992e+04,1.370584,1.508537e+04,1.349821,8.217648e+04,1.509689,9.914887e+03,1.460773,1.373760e+04,1.482101,9.409606e+04,1.360288,6.022463e+04,1.368946,2.086329e+04,1.144399,4.095916e+05,1.375125,2.834511e+04,1.475278,5753.950736,1.478189,8745.676887,1.492889,2.626450e+04,1.405662,2.285452e+04,1.426399,1.657403e+05,1.374757,7.501528e+03,1.522649,4.413002e+04,0.941273,0.449247,5.438894,0.468163,0.640297,2.005182,1.89422,1.676886,2.036558,1.451165,1.998749,1.779602,1.971593,1.845983,1.724866,0.761658,0.497810,0.272060,0.369639,0.323322,0.453943,0.247465,0.406407,0.337715,0.306465,0.469888,0.239539,0.166078,0.401989,0.226942,0.488639,0.222964,0.260499
min,0.000000,1.000000,2.500000e+01,1.000000,2.500000e+01,1.000000,25.000000,1.000000,2.600000e+01,1.000000,2.500000e+01,1.000000,2.500000e+01,1.000000,2.500000e+01,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,2.500000e+01,1.000000,2.500000e+01,1.000000,25.000000,1.000000,25.000000,1.000000,0.000000e+00,1.000000,0.000000e+00,1.000000,2.500000e+01,1.000000,2.500000e+01,1.000000,2.600000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11403.000000,1.000000,4.040000e+02,1.000000,8.760000e+02,2.000000,651.000000,1.000000,6.810000e+02,1.000000,8.360000e+02,1.000000,5.050000e+02,1.000000,5.490000e+02,1.000000,1.081000e+03,2.000000,1.257000e+03,2.000000,9.040000e+02,3.000000,8.600000e+02,4.000000,8.060000e+02,2.000000,6.510000e+02,1.000000,430.000000,2.000000,638.000000,2.000000,7.050000e+02,2.000000,4.620000e+02,2.000000,7.950000e+02,2.000000,5.290000e+0

In [6]:
# 불균형 분포인지 확인
print(vote_df['voted'].value_counts())
unsatisfied_cnt = vote_df[vote_df['voted'] == 1].voted.count()
total_cnt = vote_df.voted.count()
print("unsatisfied 비율은 {0:.2f}".format(unsatisfied_cnt / total_cnt))

0    19918
1    16507
Name: voted, dtype: int64
unsatisfied 비율은 0.45


선거 투표 여부
- 0 = No
- 1 = Yes

In [7]:
# NA값 확인
print(vote_df.isna().sum())

index         0
QaA           0
QaE           0
QbA           0
QbE           0
QcA           0
QcE           0
QdA           0
QdE           0
QeA           0
QeE           0
QfA           0
QfE           0
QgA           0
QgE           0
QhA           0
QhE           0
QiA           0
QiE           0
QjA           0
QjE           0
QkA           0
QkE           0
QlA           0
QlE           0
QmA           0
QmE           0
QnA           0
QnE           0
QoA           0
QoE           0
QpA           0
QpE           0
QqA           0
QqE           0
QrA           0
QrE           0
QsA           0
QsE           0
QtA           0
QtE           0
age_group     0
education     0
engnat        0
familysize    0
gender        0
hand          0
married       0
race          0
religion      0
tp01          0
tp02          0
tp03          0
tp04          0
tp05          0
tp06          0
tp07          0
tp08          0
tp09          0
tp10          0
urban         0
voted         0
wf_01   

### 결손값 처리

In [8]:
# 0값을 검사할 피처명 리스트 객체 설정
zero_features = ['education', 'engnat', 'hand', 'urban']
for i in range(97, 117):  # 질문을 답할 때까지의 시간
    s = 'Q' + chr(i) + 'E'
    zero_features.append(s)

# 전체 데이터 건수
total_count = vote_df['education'].count()
print("전체 데이터 건수는 ", total_count)
print()

# 피처별로 반복하면서 데이터 값이 0인 데이터 건수 추출하고 퍼센트 계산
for feature in zero_features:
    zero_count = vote_df[vote_df[feature] == 0][feature].count()
    print('{0} 0 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature, zero_count, 100*zero_count/total_count))
print()
    
# tp 범위 검사
tp_features = []
for i in range(1, 11):
    s = 'tp' + chr(48+(i//10)) + chr(48+(i%10))
    tp_features.append(s)
    
# 피처별로 반복하면서 데이터 값이 01~07을 벗어난 데이터 건수 추출하고 퍼센트 계산
for feature in tp_features:
    cnt = vote_df[vote_df[feature] < 1][feature].count() + vote_df[vote_df[feature] > 7][feature].count()
    print('{0} 건수는 {1}, 퍼센트는 {2:.2f} %'.format(feature, cnt, 100*cnt/total_count))
print()

print("value > 7\tvalue == 7\tvalue < 0")
for feature in tp_features:
    cnt1 = vote_df[vote_df[feature] > 7][feature].count()
    cnt2 = vote_df[vote_df[feature] == 0][feature].count() # tp__ 데이터 결손값이 모두 0임을 확인
    cnt3 = vote_df[vote_df[feature] < 0][feature].count()
    print("%d\t\t%d\t\t%d" % (cnt1, cnt2, cnt3))
    
zero_features = zero_features + tp_features


전체 데이터 건수는  36425

education 0 건수는 432, 퍼센트는 1.19 %
engnat 0 건수는 63, 퍼센트는 0.17 %
hand 0 건수는 135, 퍼센트는 0.37 %
urban 0 건수는 258, 퍼센트는 0.71 %
QaE 0 건수는 0, 퍼센트는 0.00 %
QbE 0 건수는 0, 퍼센트는 0.00 %
QcE 0 건수는 0, 퍼센트는 0.00 %
QdE 0 건수는 0, 퍼센트는 0.00 %
QeE 0 건수는 0, 퍼센트는 0.00 %
QfE 0 건수는 0, 퍼센트는 0.00 %
QgE 0 건수는 0, 퍼센트는 0.00 %
QhE 0 건수는 1, 퍼센트는 0.00 %
QiE 0 건수는 1, 퍼센트는 0.00 %
QjE 0 건수는 1, 퍼센트는 0.00 %
QkE 0 건수는 2, 퍼센트는 0.01 %
QlE 0 건수는 0, 퍼센트는 0.00 %
QmE 0 건수는 0, 퍼센트는 0.00 %
QnE 0 건수는 0, 퍼센트는 0.00 %
QoE 0 건수는 0, 퍼센트는 0.00 %
QpE 0 건수는 1, 퍼센트는 0.00 %
QqE 0 건수는 1, 퍼센트는 0.00 %
QrE 0 건수는 0, 퍼센트는 0.00 %
QsE 0 건수는 0, 퍼센트는 0.00 %
QtE 0 건수는 0, 퍼센트는 0.00 %

tp01 건수는 3822, 퍼센트는 10.49 %
tp02 건수는 4356, 퍼센트는 11.96 %
tp03 건수는 9348, 퍼센트는 25.66 %
tp04 건수는 4414, 퍼센트는 12.12 %
tp05 건수는 12682, 퍼센트는 34.82 %
tp06 건수는 7148, 퍼센트는 19.62 %
tp07 건수는 7858, 퍼센트는 21.57 %
tp08 건수는 2547, 퍼센트는 6.99 %
tp09 건수는 7027, 퍼센트는 19.29 %
tp10 건수는 1158, 퍼센트는 3.18 %

value > 7	value == 7	value < 0
0		3822		0
0		4356		0
0		9348		0
0		4414		0
0		126

In [9]:
# zero_features 리스트 내부에 저장된 개별 피처들에 대해 0값을 평균 값으로 대체
vote_df[zero_features] = vote_df[zero_features].replace(0, vote_df[zero_features].mean())

### 이상값 처리

In [10]:
# def get_outlier(df=None, column=None, weight=1.5):
    